<a href="https://colab.research.google.com/github/daniela2001v2-png/medflow/blob/main/medflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MedFlow MVP - Google Colab

* Hecho por: Yeinmy Daniela Morales Barrera - ymoral35@estudiante.ibero.edu.co
* Fecha: 30 de Octubre de 2025


INSTRUCCIONES:
1. Runtime > Cambie el tipo de tiempo de ejecución (Runtime) > T4 GPU
2. Ejecuta CELDA 1, luego CELDA 2, etc. en orden
3. La última celda lanzará la interfaz del usuario


In [1]:
"""
Instala las dependencias necesarias para el proyecto MedFlow.

Las dependencias incluyen:
- transformers: Para trabajar con modelos de Hugging Face.
- accelerate: Para optimizar la inferencia del modelo en GPU.
- gradio: Para crear la interfaz de usuario.
- pillow: Para manejo de imágenes.
- requests: Para realizar peticiones HTTP.
- torch: Para operaciones con tensores y uso de GPU.
- sphinx, sphinx-rtd-theme, nbsphinx, sphinx-autodoc-typehints: Para la generación de documentación.
"""
print("📦 Instalando dependencias...")

# Instalar las dependencias principales en modo silencioso (-q)
!pip install -q transformers==4.50.0 accelerate gradio pillow requests torch
# Instalar dependencias para la generación de documentación
!pip install sphinx sphinx-rtd-theme nbsphinx sphinx-autodoc-typehints
print("✅ Listo!")

📦 Instalando dependencias...
✅ Listo!


In [ ]:
"""
Importa las librerías y módulos necesarios para la aplicación MedFlow.

Se importan:
- torch: Para operaciones con tensores y manejo de GPU.
- AutoProcessor, AutoModelForImageTextToText de transformers: Para cargar el modelo y procesador de Hugging Face.
- Image de PIL (Pillow): Para el manejo de imágenes.
- gradio: Para la construcción de la interfaz de usuario.
- time: Para medir el tiempo de procesamiento.
- traceback: Para obtener información detallada de errores.
"""
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText
from PIL import Image
import gradio as gr
import time
import traceback

In [ ]:
def validate_gpu_type() -> bool:
  """
  Valida el tipo de GPU a usar dentro del entorno de Google Colab.

  Verifica si CUDA está disponible y, si es así, imprime información sobre la GPU.
  Limpia la caché de CUDA para liberar memoria.

  Args:
    None

  Returns:
    bool: True si CUDA está disponible, False en caso contrario.
  """
  print("\n🔍 Verificando GPU...")
  print(f"CUDA disponible: {torch.cuda.is_available()}")
  if torch.cuda.is_available():
      print(f"GPU: {torch.cuda.get_device_name(0)}")
      print(f"Memoria: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
      # Limpiar cache CUDA
      torch.cuda.empty_cache()
      return torch.cuda.is_available()
  else:
      print("⚠️ No hay GPU. Ve a Runtime > Change runtime type > T4 GPU")
      return False


validate_gpu_type()

In [ ]:
# Validar si se ha configurado CUDA (GPU)
is_cuda_setted = validate_gpu_type()
# Definir el ID del modelo a utilizar de Hugging Face
MODEL_ID = "google/medgemma-4b-it"
# Configurar el dispositivo de procesamiento ('cuda' si hay GPU, 'cpu' en caso contrario)
device = "cuda" if is_cuda_setted else "cpu"


def init_medflow_model():
  """
  Inicializa el modelo de MedFlow (Med-GEMMA 4B Multimodal).

  Descarga y carga el modelo y el procesador asociados desde Hugging Face.
  Configura el modelo para usar bfloat16 y device_map="auto" para optimizar el uso de memoria y recursos.

  Args:
    None

  Returns:
    tuple: Una tupla conteniendo el procesador y el modelo cargados.
           Retorna (None, None) si ocurre un error durante la carga.
  """

  print(f"\n🏥 Iniciando MedFlow...")
  print(f"⚙️ Dispositivo: {device}")
  print(f"📥 Descargando modelo (5-10 min primera vez)...\n")

  try:
      # Cargar el procesador asociado al modelo
      processor = AutoProcessor.from_pretrained(MODEL_ID)

      # Cargar el modelo pre-entrenado
      model = AutoModelForImageTextToText.from_pretrained(
          MODEL_ID,
          torch_dtype=torch.bfloat16,  # Usar bfloat16 para eficiencia y compatibilidad con GPU
          device_map="auto",  # Distribuir el modelo automáticamente entre dispositivos disponibles
          low_cpu_mem_usage=True # Reducir el uso de CPU al cargar el modelo
      )

      print("✅ Modelo cargado exitosamente!\n")
      return processor, model

  except Exception as e:
      print(f"❌ Error cargando modelo: {e}")
      traceback.print_exc()
      return None, None

In [ ]:
# Cargar el procesador y el modelo utilizando la función init_medflow_model
processor, model = init_medflow_model()

# Verificar si el modelo y procesador se cargaron correctamente
if processor is None or model is None:
    print("🔴 No se pudo cargar el modelo MedFlow. Verifique los mensajes de error anteriores.")
else:
    print("🟢 Modelo MedFlow listo para usar.")

In [ ]:
"""
Define la estructura y roles de los componentes de la interfaz de usuario (UI) de Gradio.

Este diccionario `ui_components` mapea nombres lógicos de componentes a sus
tipos, roles (Input, Output, Action, Container, Display, Layout, etc.), y una
breve descripción de su propósito.

El diccionario `connected_function` describe la función de backend que se
ejecuta al interactuar con la UI, el evento que la dispara, y los componentes
de entrada y salida involucrados.

Returns:
    None: Esta celda define variables globales y no retorna un valor.
"""
# Diccionario que identifica los componentes de la UI de Gradio, su tipo, rol y descripción.
ui_components = {
    "imagen_input": {"type": "gr.Image", "role": "Input", "description": "Medical image input (X-ray, CT, etc.)"},
    "tipo_analisis": {"type": "gr.Dropdown", "role": "Input", "description": "Type of analysis (General Description, Pathological Findings, Structured Report, Differential Diagnosis)"},
    "idioma": {"type": "gr.Radio", "role": "Input", "description": "Report language (Spanish or English)"},
    "procesar_btn": {"type": "gr.Button", "role": "Action", "description": "Button to trigger image analysis"},
    "limpiar_btn": {"type": "gr.ClearButton", "role": "Action", "description": "Button to clear the image input"},
    "reporte_output": {"type": "gr.Textbox", "role": "Output", "description": "Generated medical report"},
    "status_output": {"type": "gr.Textbox", "role": "Output", "description": "Processing status"},
    "metadata_output": {"type": "gr.Markdown", "role": "Output", "description": "Processing metadata"},
    "demo": {"type": "gr.Blocks", "role": "Container", "description": "Main container for the Gradio application"},
    "Markdown": {"type": "gr.Markdown", "role": "Display", "description": "Used for titles, descriptions, and instructions"},
    "Row": {"type": "gr.Row", "role": "Layout", "description": "Arranges components horizontally"},
    "Column": {"type": "gr.Column", "role": "Layout", "description": "Arranges components vertically"},
    "Accordion": {"type": "gr.Accordion", "role": "Layout", "description": "Collapsible section for content"},
    "Examples": {"type": "gr.Examples", "role": "Input/Demonstration", "description": "Provides example inputs"},
}

# Diccionario que identifica la función conectada a un evento de la UI y sus interacciones (entradas/salidas).
connected_function = {
    "name": "analizar_imagen",
    "triggered_by": "procesar_btn.click",
    "inputs": ["imagen_input", "tipo_analisis", "idioma"],
    "outputs": ["reporte_output", "metadata_output", "status_output"]
}

# Imprimir la información de los componentes identificados y la función conectada.
print("Identified UI Components:")
for component, details in ui_components.items():
    print(f"- {component}: Type={details['type']}, Role={details['role']}, Description={details['description']}")

print("\nConnected Function:")
print(f"- Name: {connected_function['name']}")
print(f"- Triggered by: {connected_function['triggered_by']}")
print(f"- Inputs: {connected_function['inputs']}")
print(f"- Outputs: {connected_function['outputs']}")

In [ ]:
import requests
import os

def create_input_section():
    """
    Crea y configura la sección de entrada de la interfaz de usuario.

    Esta sección incluye el componente de carga de imagen, el menú desplegable
    para el tipo de análisis, los botones de radio para el idioma y los botones
    de procesar y limpiar.

    Returns:
        tuple: Una tupla conteniendo:
            - gr.Column: El layout de columna de Gradio para la sección de entrada.
            - gr.Image: El componente de entrada de imagen.
            - gr.Dropdown: El menú desplegable para el tipo de análisis.
            - gr.Radio: Los botones de radio para el idioma.
            - gr.Button: El botón para iniciar el análisis.
            - gr.ClearButton: El botón para limpiar la entrada de imagen.
    """
    with gr.Column(scale=1) as input_col:
        gr.Markdown("### 📤 Entrada de Datos")

        imagen_input = gr.Image(
            type="pil",
            label="📷 Imagen Médica (Rayos X, TC, etc.)",
            height=350
        )

        gr.Markdown("**Configuración:**")

        tipo_analisis = gr.Dropdown(
            choices=[
                "Descripción General",
                "Hallazgos Patológicos",
                "Reporte Estructurado",
                "Diagnóstico Diferencial"
            ],
            value="Reporte Estructurado",
            label="🔍 Tipo de Análisis",
            info="Selecciona el enfoque del análisis"
        )

        idioma = gr.Radio(
            choices=["Español", "Inglés"],
            value="Español",
            label="🌐 Idioma del Reporte"
        )

        with gr.Row():
            procesar_btn = gr.Button(
                "🚀 Analizar Imagen",
                variant="primary",
                size="lg",
                scale=2
            )
            limpiar_btn = gr.ClearButton(
                components=[imagen_input],
                value="🗑️ Limpiar",
                size="lg",
                scale=1
            )
    return input_col, imagen_input, tipo_analisis, idioma, procesar_btn, limpiar_btn


def create_output_section():
    """
    Crea y configura la sección de salida de la interfaz de usuario.

    Esta sección incluye el cuadro de texto para mostrar el reporte médico
    generado y el cuadro de texto para mostrar el estado del procesamiento.

    Returns:
        tuple: Una tupla conteniendo:
            - gr.Column: El layout de columna de Gradio para la sección de salida.
            - gr.Textbox: El cuadro de texto para el reporte generado.
            - gr.Textbox: El cuadro de texto para el procesamiento.
    """
    with gr.Column(scale=1) as output_col:
        gr.Markdown("### 📋 Resultado del Análisis")

        reporte_output = gr.Markdown(
            value="*El reporte aparecerá aquí después de procesar la imagen...*",
            label="Reporte Médico Generado",
            # Markdown no tiene 'lines', pero puedes controlar altura con CSS
        )

        status_output = gr.Textbox(
            label="Estado del Proceso",
            lines=1,
            interactive=False,
            show_copy_button=False
        )

    return output_col, reporte_output, status_output


def create_metadata_accordion():
    """
    Crea y configura el acordeón de metadatos.

    Esta sección de acordeón se utiliza para mostrar metadatos de procesamiento
    utilizando un componente Markdown.

    Returns:
        tuple: Una tupla conteniendo:
            - gr.Accordion: El componente acordeón de Gradio.
            - gr.Markdown: El componente Markdown para mostrar metadatos.
    """
    with gr.Accordion("📊 Metadatos de Procesamiento", open=False) as metadata_acc:
        metadata_output = gr.Markdown()
    return metadata_acc, metadata_output

def create_guide_accordion():
    """
    Crea y configura el acordeón de la guía de uso.

    Esta sección de acordeón contiene instrucciones detalladas, tipos de imágenes
    soportadas, especificaciones técnicas, limitaciones y fases futuras del
    proyecto dentro de un componente Markdown.

    Returns:
        gr.Accordion: El componente acordeón de Gradio que contiene la guía de uso.
    """
    guide_markdown = """
        ### 🎯 Instrucciones Detalladas:

        1. **Preparar Imagen:**
           - Formatos soportados: JPG, PNG, DICOM
           - Tamaño recomendado: < 5MB
           - Resolución óptima: 512x512 a 2048x2048 píxeles

        2. **Cargar Imagen:**
           - Haz clic en el área de carga
           - Selecciona archivo o arrastra la imagen
           - Espera a que se visualice

        3. **Configurar Análisis:**
           - **Descripción General:** Identificación de estructuras anatómicas
           - **Hallazgos Patológicos:** Detección específica de anomalías
           - **Reporte Estructurado:** Formato radiológico estándar (TÉCNICA/HALLAZGOS/IMPRESIÓN)
           - **Diagnóstico Diferencial:** Lista de posibles diagnósticos

        4. **Ejecutar:**
           - Presiona "Analizar Imagen"
           - Espera 10-30 segundos según tamaño de imagen
           - Revisa el reporte generado

        ### 📚 Tipos de Imágenes Soportadas:

        | Modalidad | Ejemplos |
        |-----------|----------|
        | **Radiología** | Rayos X tórax, abdomen, extremidades |
        | **Tomografía** | TC cerebral, torácica, abdominal |
        | **Dermatología** | Lesiones cutáneas, erupciones |
        | **Patología** | Histopatología, biopsias |
        | **Oftalmología** | Fondo de ojo, retinografías |

        ### ⚙️ Especificaciones Técnicas:

        - **Modelo Base:** Med-GEMMA 4B Multimodal
        - **Arquitectura:** Gemma 3 + SigLIP Image Encoder
        - **Entrenamiento:** Datos médicos (MIMIC-CXR, PathMCQA, etc.)
        - **Precisión Reportada:** 88.9 macro F1 en clasificación MIMIC-CXR
        - **Hardware:** Google Colab T4 GPU (16GB VRAM)
        - **Framework:** PyTorch + Hugging Face Transformers

        ### 🔬 Limitaciones del MVP:

        - No reemplaza el juicio clínico profesional
        - Optimizado para imágenes en inglés (dataset base)
        - Sin fine-tuning para datos colombianos específicos
        - Sin integración con sistemas hospitalarios PACS
        - Sin validación clínica por radiólogos certificados

        ### 📈 Próximas Fases del Proyecto:

        1. **Fase 2:** Fine-tuning con datos colombianos
        2. **Fase 3:** Validación con radiólogos locales
        3. **Fase 4:** Integración PACS/RIS
        4. **Fase 5:** Cumplimiento normativo (Ley 1581 de 2012)
        """
    with gr.Accordion("📖 Guía de Uso Completa", open=False) as guide_acc:
        gr.Markdown(guide_markdown)
    return guide_acc

def create_examples_section(imagen_input, tipo_analisis, idioma):
    """
    Crea y configura la sección de ejemplos.

    Esta sección proporciona ejemplos de entrada en los que los usuarios pueden hacer
    clic para poblar los campos de entrada y ver cómo funciona la aplicación.

    Args:
        imagen_input (gr.Image): El componente de entrada de imagen para vincular ejemplos.
        tipo_analisis (gr.Dropdown): El menú desplegable del tipo de análisis para vincular ejemplos.
        idioma (gr.Radio): Los botones de radio del idioma para vincular ejemplos.

    Returns:
        gr.Examples: El componente Gradio Examples.
    """
    # URL de la imagen de ejemplo
    example_image_url = "https://upload.wikimedia.org/wikipedia/commons/c/c8/Chest_Xray_PA_3-8-2010.png"
    # Nombre del archivo local para la imagen de ejemplo
    example_image_filename = "chest_xray_example.png"

    # Descargar la imagen si no existe localmente
    if not os.path.exists(example_image_filename):
        try:
            print(f"Descargando imagen de ejemplo desde: {example_image_url}")
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            }
            response = requests.get(example_image_url, headers=headers, stream=True, timeout=10)
            response.raise_for_status()
            with open(example_image_filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"Imagen de ejemplo descargada a: {example_image_filename}")
        except requests.exceptions.RequestException as e:
            print(f"Error al descargar la imagen de ejemplo: {e}")
            # Si falla la descarga, usar una lista de ejemplos vacía
            examples_data = []
            print("No se pudieron cargar los ejemplos. La sección de ejemplos estará vacía.")
            examples_section = gr.Examples(
                examples=examples_data,
                inputs=[imagen_input, tipo_analisis, idioma],
                label="📚 Ejemplo de Uso: Rayos X de Tórax (No disponible)"
            )
            return examples_section # Retornar aquí si falla la descarga

    # Si la descarga fue exitosa o la imagen ya existía, definir los ejemplos con la ruta local
    examples_data = [
        [example_image_filename, "Reporte Estructurado", "Español"],
    ]

    examples_section = gr.Examples(
        examples=examples_data,
        inputs=[imagen_input, tipo_analisis, idioma],
        label="📚 Ejemplo de Uso: Rayos X de Tórax"
    )
    return examples_section

def create_footer():
    """
    Crea y configura el markdown del pie de página.

    Esta sección contiene información de contacto, enlace al repositorio y
    detalles del proyecto.

    Returns:
        gr.Markdown: El componente Gradio Markdown para el pie de página.
    """
    footer_markdown = """
    ---
    ### 📞 Información del Proyecto

    **Contacto:** ymoral35@estudiante.ibero.edu.co
    **Repositorio:** [Documentación completa disponible próximamente]
    **Versión:** MVP 1.0 (Octubre 2025)

    *Desarrollado como parte del programa de Práctica Profesional (PICD) en Ingeniería en Ciencia de Datos*
    """
    return gr.Markdown(footer_markdown)

In [ ]:
import time
import traceback

def analizar_imagen(imagen, tipo_analisis="Reporte Estructurado", idioma="Español"):
    """
    Analiza una imagen médica utilizando el modelo Med-GEMMA.

    Procesa la imagen de entrada junto con un prompt basado en el tipo de análisis y idioma
    seleccionados. Genera un reporte médico estructurado o descriptivo.

    Args:
        imagen (PIL.Image.Image): La imagen médica a analizar.
        tipo_analisis (str, optional): El tipo de análisis a realizar.
            Puede ser "Descripción General", "Hallazgos Patológicos",
            "Reporte Estructurado", o "Diagnóstico Diferencial".
            Por defecto es "Reporte Estructurado".
        idioma (str, optional): El idioma en el que se generará el reporte.
            Puede ser "Español" o "Inglés". Por defecto es "Español".

    Returns:
        tuple: Una tupla conteniendo:
            - str: El reporte médico generado o un mensaje de error.
            - str: Metadatos del procesamiento (tiempo, modelo, GPU, etc.).
            - str: El estado del proceso (Completado, Error).
    """

    # Validar si se ha cargado una imagen
    if imagen is None:
        return "❌ Por favor carga una imagen primero", "", "Error: Sin imagen"

    try:
        inicio = time.time()

        # Prompts en español para los diferentes tipos de análisis
        prompts = {
            "Descripción General": "Describe esta imagen médica identificando las estructuras anatómicas visibles.",
            "Hallazgos Patológicos": "Identifica cualquier hallazgo patológico o anormal en esta imagen médica.",
            "Reporte Estructurado": "Genera un reporte médico estructurado con: TÉCNICA, HALLAZGOS e IMPRESIÓN.",
            "Diagnóstico Diferencial": "Proporciona un diagnóstico diferencial basado en los hallazgos visibles."
        }

        # Obtener el prompt adecuado según el tipo de análisis, con fallback a "Reporte Estructurado"
        prompt = prompts.get(tipo_analisis, prompts["Reporte Estructurado"])

        # Preparar los mensajes en el formato de chat para el modelo
        messages = [
            {
                "role": "system",
                "content": [{"type": "text", "text": "Eres un radiólogo experto especializado en interpretación de imágenes médicas."}]
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image", "image": imagen}
                ]
            }
        ]

        # Aplicar el template de chat y tokenizar la entrada
        # Primero aplicar template sin tokenizar para obtener el texto completo
        text_inputs = processor.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=False  # Importante: primero sin tokenizar
        )

        # Luego tokenizar por separado incluyendo la imagen
        inputs = processor(
            text=text_inputs,
            images=imagen,
            return_tensors="pt",
            padding=True
        )

        # Mover los tensores de entrada al dispositivo de procesamiento (GPU o CPU)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Obtener la longitud de los tokens de entrada para decodificar solo la respuesta
        input_len = inputs["input_ids"].shape[-1]

        # Generar la respuesta del modelo
        print(f"🤖 Generando reporte...")

        # Usar torch.no_grad() para deshabilitar el cálculo de gradientes durante la inferencia
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=350,  # Número máximo de tokens a generar
                do_sample=False,  # Deshabilitar muestreo para una salida determinística
                num_beams=1, # Usar beam search con 1 haz (equivalente a greedy search)
                pad_token_id=processor.tokenizer.pad_token_id, # ID del token de padding
                eos_token_id=processor.tokenizer.eos_token_id # ID del token de fin de secuencia
            )

        # Decodificar solo los tokens generados por el modelo (excluyendo los tokens de entrada)
        generated_tokens = outputs[0][input_len:]
        reporte = processor.decode(generated_tokens, skip_special_tokens=True)

        tiempo = time.time() - inicio

        # Agregar un disclaimer médico al reporte
        disclaimer = """

⚠️ DISCLAIMER MÉDICO:
Este reporte es generado por IA con propósito educativo y demostrativo únicamente.
NO debe utilizarse para decisiones clínicas sin validación por profesionales médicos.
Proyecto académico - Corporación Universitaria Iberoamericana.
"""

        reporte_final = reporte + disclaimer

        # Generar metadatos del procesamiento
        metadata = f"""
📊 **Información de Procesamiento:**
- ⏱️ Tiempo: {tiempo:.2f} segundos
- 🤖 Modelo: Med-GEMMA 4B (Google Health AI)
- 💻 GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}
- 📝 Tokens generados: {len(generated_tokens)}
- 🔧 Tipo análisis: {tipo_analisis}
"""

        # Establecer el estado de completado
        status = f"✅ Completado exitosamente en {tiempo:.2f}s"

        return reporte_final, metadata, status

    except Exception as e:
        # Capturar y formatear cualquier error que ocurra durante el procesamiento
        error_msg = f"""
❌ ERROR durante el análisis:

{str(e)}

**Posibles soluciones:**
1. Verifica que tengas GPU habilitada (Runtime > Change runtime type)
2. Reinicia el runtime (Runtime > Restart runtime)
3. Intenta con una imagen más pequeña
4. Si persiste, puede ser límite de memoria - prueba cerrar otras pestañas
"""
        print(f"\n❌ Error completo:\n{traceback.format_exc()}")
        # Retornar mensajes de error y estado
        return error_msg, "Error en procesamiento", "❌ Error"

In [ ]:
# Definir el CSS para la interfaz de Gradio.
css = """
.gradio-container {
    max-width: 1400px !important;
    margin: auto;
}
h1 {
    text-align: center;
    color: #2563eb;
}
"""

# Iniciar un contexto gr.Blocks para construir la interfaz.
with gr.Blocks(title="MedFlow MVP", theme=gr.themes.Soft(), css=css) as demo:

    # Agregar el título principal y la descripción en formato Markdown.
    gr.Markdown("""
    # 🏥 MedFlow - Producto Mínimo Viable
    ## Sistema de Interpretación Automatizada de Imágenes Médicas

    **Proyecto de Práctica Profesional - Ingeniería en Ciencia de Datos**
    **Desarrollado por:** Yeinmy Daniela Morales Barrera
    **Institución:** Corporación Universitaria Iberoamericana
    **Modelo:** Med-GEMMA 4B (Google Health AI)

    ---
    """)

    # Crear un layout horizontal para las secciones de entrada y salida.
    with gr.Row():
        # Agregar la columna de entrada y capturar los componentes retornados.
        input_col, imagen_input, tipo_analisis, idioma, procesar_btn, limpiar_btn = create_input_section()

        # Agregar la columna de salida y capturar los componentes retornados.
        output_col, reporte_output, status_output = create_output_section()

    # Agregar la sección de metadatos (acordeón).
    metadata_acc, metadata_output = create_metadata_accordion()

    # Agregar la sección de guía de uso (acordeón).
    guide_acc = create_guide_accordion()

    # Agregar la sección de ejemplos.
    examples_section = create_examples_section(imagen_input, tipo_analisis, idioma)

    # Agregar el pie de página.
    footer = create_footer()

    # Conectar el evento click del botón de procesar con la función analizar_imagen.
    procesar_btn.click(
        fn=analizar_imagen,
        inputs=[imagen_input, tipo_analisis, idioma],
        outputs=[reporte_output, metadata_output, status_output]
    )

# Agregar las declaraciones print para lanzar la aplicación.
print("\n" + "="*70)
print("🚀 LANZANDO MEDFLOW MVP")
print("="*70)
print("\n✅ Todo configurado correctamente")
print("📱 La interfaz se abrirá automáticamente")
print("🔗 Compartirás un link público temporal (válido 72h)")
print("\n⚠️ IMPORTANTE: No cierres esta pestaña mientras uses la app\n")

# Lanzar la demo de Gradio.
demo.launch(
    share=True,  # Habilitar enlace público compartido
    debug=True,  # Habilitar modo debug para ver errores detallados
    show_error=True, # Mostrar errores en la UI
    server_name="0.0.0.0", # Escuchar en todas las interfaces de red
    server_port=7860 # Usar el puerto 7860
)